#Files needed
* CNN_CIFAR10_net3.h5
* adv_de_cnn_cifar_ssim
* subset_cifar

# Import Libraries

In [0]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from scipy.stats import cauchy 
from scipy import stats
import operator
import random
from sklearn.metrics import accuracy_score
from skimage.measure import compare_ssim as ssim
import pickle
import cv2
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, Activation

#Set seeds
random.seed(0)
np.random.seed(0)

# Get the data from pickle files

In [0]:

original = None
adversarial = None 
original_y = None
adversarial_y = None
dump = None
adversarial_rbf = None
adversarial_rbf_y  = None
adversarial_cnn = None
adversarial_cnn_y  = None

with open('subset_cifar', 'rb') as f:
    original = pickle.load(f)
    adversarial= pickle.load(f)
    original_y = pickle.load(f)
    adversarial_y = pickle.load(f)

with open('adv_de_cnn_cifar_ssim', 'rb') as f:
    adversarial_cnn = pickle.load(f)
    for i in range(3):
      dump = pickle.load(f)
    adversarial_cnn_y= pickle.load(f)

In [0]:
original.shape

(914, 32, 32, 1)

In [0]:
adversarial.shape

(133, 32, 32, 1)

In [0]:
adversarial_cnn.shape

(133, 32, 32, 1)

# Calculate PSNR and SSIM and Euclidean

In [0]:
psnrs_cnn = []
for i in range(adversarial.shape[0]):
  psnrs_cnn.append(cv2.PSNR(adversarial[i].reshape(32,32), adversarial_cnn[i].reshape(32,32)))

In [0]:
#double check GeeksForGeeks psnr
print(np.around(np.mean(psnrs_cnn),3), "+-" ,np.around(np.std(psnrs_cnn),3),"(" ,np.around(np.min(psnrs_cnn),3), ",",
                                                                                        np.around(np.max(psnrs_cnn),3),")")

27.659 +- 6.281 ( 6.746 , 36.16 )


In [0]:
ssims_cnn = []
for i in range(adversarial.shape[0]):
  ssims_cnn.append(ssim(adversarial[i].reshape(32,32), adversarial_cnn[i].reshape(32,32)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
print(np.around(np.mean(ssims_cnn),3), "+-" ,np.around(np.std(ssims_cnn),3),"(" ,np.around(np.min(ssims_cnn),3), ",",
                                                                                        np.around(np.max(ssims_cnn),3),")")

0.851 +- 0.151 ( 0.243 , 0.985 )


In [0]:
# https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy
def euclidean(s):
    # pass s=a-b as argument to the function
    return np.linalg.norm(s)

In [0]:
eucs_cnn = []
for i in range(adversarial.shape[0]):
  eucs_cnn.append(euclidean(adversarial[i].reshape(32,32) -  adversarial_cnn[i].reshape(32,32)))

In [0]:
print(np.around(np.mean(eucs_cnn),3), "+-" ,np.around(np.std(eucs_cnn),3),"(" ,np.around(np.min(eucs_cnn),3), ",",
                                                                                        np.around(np.max(eucs_cnn),3),")")

464.626 +- 487.014 ( 126.963 , 3752.988 )


# Prepare Date

In [0]:
#Normalize images
original_sc = original        /255.0
adversarial_sc = adversarial     /255.0 
adversarial_cnn_sc = adversarial_cnn /255.0

# Predict Using CNN

In [0]:
from keras.models import load_model
model = load_model("CNN_CIFAR10_net3.h5")

In [0]:
#Get predictions
cnn_pred_original = model.predict(original_sc)
cnn_pred_adversarial = model.predict(adversarial_sc)
cnn_pred_adversarial_cnn = model.predict(adversarial_cnn_sc)

In [0]:
print("Accuracy on original: ",accuracy_score(original_y,np.argmax(cnn_pred_original,axis=1)))
print("Accuracy on adversarial: ",accuracy_score(adversarial_y,np.argmax(cnn_pred_adversarial,axis=1)))
print("Accuracy on adversarial_cnn: ",accuracy_score(adversarial_y,np.argmax(cnn_pred_adversarial_cnn,axis=1)))

Accuracy on original:  0.824945295404814
Accuracy on adversarial:  0.8195488721804511
Accuracy on adversarial_cnn:  0.0


In [0]:
print("Accuracy on subset: ",accuracy_score(np.append(original_y,adversarial_y,axis=0),np.argmax(np.append(cnn_pred_original,cnn_pred_adversarial,axis=0),axis=1)))
print("Accuracy on subset with adversarial examples (CNN): ",accuracy_score(np.append(original_y,adversarial_y,axis=0),np.argmax(np.append(cnn_pred_original,cnn_pred_adversarial_cnn,axis=0),axis=1)))

Accuracy on subset:  0.8242597898758357
Accuracy on subset with adversarial examples (CNN):  0.720152817574021
